In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/home/airazb/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-28 19:00:00| 134|193.61000000000007|            17|
|2020-01-22 19:00:00|  65| 657.0300000000001|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|229.39999999999998|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|165.90000000000003|             9|
|2020-01-17 10:00:00|  41| 638.2699999999999|            49|
|2020-01-02 11:00:00|  26|             198.6|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|148.89000000000001|            13|
|2020-01-04 20:00:00|  25| 369.5700000000001|            23|
|2020-01-22 10:00:00|  7

In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [11]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [14]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [15]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [18]:
df_join = spark.read.parquet('data/report/revenue/total')

In [19]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [20]:
df_zones = spark.read.parquet('zones/')

In [21]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [23]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

AnalysisException: path file:/home/airazb/data-engineering-zoomcamp/week_5_batch_processing/code/tmp/revenue-zones already exists.